In [1]:
import numpy as np
import tensorflow as tf
# from models import Input, AspectOutput
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D,GRU
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Concatenate
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
import json
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [2]:
pre_bert = np.load(r"C:\Users\acer\Documents\hoc tap\khóa luận\mebe_domain\phobert_embed.npy", allow_pickle=True)


In [19]:
pre_bert[4].shape

(10, 768)

In [4]:
max_len = 50


In [5]:
pre_bert_padding = []
for sen in pre_bert:
    emb = np.zeros((max_len, 768))
    for i in range(len(sen)):
        try:
            emb[i] = sen[i]
        except: continue
    pre_bert_padding.append(emb)

In [18]:
len(pre_bert_padding[4])

50

In [7]:
len(pre_bert)

6004

In [8]:
pre_bert_shopee = np.array(pre_bert_padding[0:3017])
pre_bert_tiki = np.array(pre_bert_padding[3017:])

In [9]:
len(pre_bert_tiki)

2987

In [10]:
with open ("../../output_shopee.txt", "r") as f:
    output_ori_shopee = np.array(json.loads(f.read()))
with open ("../../output_tiki.txt", "r") as f:
    output_ori_tiki = np.array(json.loads(f.read()))

In [11]:
X_train_shopee, X_test_shopee, y_train_ori_shopee, y_test_ori_shopee = train_test_split(pre_bert_shopee, output_ori_shopee, test_size=0.2, random_state=14)

In [12]:
len(X_train_shopee)

2413

In [13]:
y_train_shopee = np.delete(y_train_ori_shopee, 6, 1)
y_test_shopee = np.delete(y_test_ori_shopee, 6, 1)

In [14]:
f1 = tfa.metrics.F1Score(num_classes=6, average='micro', threshold=0.5)
# rlp = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',mode='min', patience=3, verbose=1, factor=0.4)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=6, verbose=1, restore_best_weights=True)
filepath = ('./weights/best_teacher.h5')
checkpoint = ModelCheckpoint(filepath, monitor='val_f1_score', verbose=1, save_best_only=True,save_weights_only=True, mode='max')

In [16]:
def get_model():
    inp = Input(shape=(max_len, 768))
#     x = Embedding(len(embed_mat), feat_vec, weights=[np.array(embed_mat)], input_length=max_len,trainable=True)(inp) # trọng số của từ sẽ không được train lại
    x = Bidirectional(LSTM(128, return_sequences=True))(inp)
    x = Conv1D(64,3,activation="relu")(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(256, activation="relu")(x)
#     convo1 = Conv1D(64,3,activation="relu")(x)
#     convo1_maxpool = GlobalMaxPool1D()(convo1)
#     convo2 = Conv1D(128,4,activation="relu")(x)
#     convo2_maxpool = GlobalMaxPool1D()(convo2)
#     convo3 = Conv1D(64,5,activation="relu")(x)
#     convo3_maxpool = GlobalMaxPool1D()(convo3)
#     concat1 = Concatenate()([convo1_maxpool, convo2_maxpool])
#     concat2 = Concatenate()([concat1, convo3_maxpool])
#     x = Dense(256, activation="relu")(concat2)
    
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    print(model.summary())

    return model
    
# model = get_model(X_train, X_test, y_train, y_test)
# model = get_model(padded_doc_shopee, X_test_tiki, output_train_shopee, y_test_tiki)
model = get_model()
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.01), metrics= [f1])
model.fit(X_train_shopee, y_train_shopee, batch_size=8, epochs=15, validation_data=(X_test_shopee, y_test_shopee), callbacks=[checkpoint])

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 50, 768)]         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 256)           918528    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 48, 64)            49216     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               16640     
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 1542      
Total params: 985,926
Trainable params: 985,926
Non-trainable params: 0
________________________________________________

KeyboardInterrupt: 

In [20]:
new_model = get_model()
new_model.load_weights('./weights/best_teacher.h5')

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 50, 768)]         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 50, 256)           918528    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 48, 64)            49216     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               16640     
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 1542      
Total params: 985,926
Trainable params: 985,926
Non-trainable params: 0
________________________________________________

In [21]:
from sklearn import metrics
def evaluate(X_test, y_test, model):    
    y_pre = model.predict(X_test)
    for thresh in np.arange(0.2,0.7,0.01):
        print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(y_test,(y_pre>thresh).astype(int), average='micro')))
    return y_pre

In [22]:
y_pre=evaluate(X_test_shopee, y_test_shopee, new_model)

threshold 0.20 f1 score:0.624
threshold 0.21 f1 score:0.630
threshold 0.22 f1 score:0.639
threshold 0.23 f1 score:0.644
threshold 0.24 f1 score:0.652
threshold 0.25 f1 score:0.659
threshold 0.26 f1 score:0.661
threshold 0.27 f1 score:0.666
threshold 0.28 f1 score:0.667
threshold 0.29 f1 score:0.675
threshold 0.30 f1 score:0.679
threshold 0.31 f1 score:0.683
threshold 0.32 f1 score:0.686
threshold 0.33 f1 score:0.682
threshold 0.34 f1 score:0.680
threshold 0.35 f1 score:0.685
threshold 0.36 f1 score:0.689
threshold 0.37 f1 score:0.686
threshold 0.38 f1 score:0.686
threshold 0.39 f1 score:0.686
threshold 0.40 f1 score:0.685
threshold 0.41 f1 score:0.683
threshold 0.42 f1 score:0.684
threshold 0.43 f1 score:0.683
threshold 0.44 f1 score:0.684
threshold 0.45 f1 score:0.682
threshold 0.46 f1 score:0.684
threshold 0.47 f1 score:0.677
threshold 0.48 f1 score:0.677
threshold 0.49 f1 score:0.669
threshold 0.50 f1 score:0.668
threshold 0.51 f1 score:0.648
threshold 0.52 f1 score:0.644
threshold 

In [23]:
thresh=0.36
y_pre_round = []
for i in range(len(y_pre)):
    y_pre_round.append(1*(y_pre[i] >= thresh))
    y_pre_round[i] = y_pre_round[i].tolist()
    if 1 not in y_pre_round[i]:
        y_pre_round[i].append(1)
    else: y_pre_round[i].append(0)
 

In [24]:
import sys
sys.path.append(r"C:\Users\acer\Documents\hoc tap\khóa luận\mebe_domain")
from modules.evaluate import cal_aspect_prf

X = cal_aspect_prf(y_test_ori_shopee.tolist(), y_pre_round, num_of_aspect=7, verbal=True)

p: [0.9487179487179487, 0.9108910891089109, 0, 0.48338368580060426, 0, 0, 0.2727272727272727]
r: [0.9319899244332494, 0.8932038834951457, 0, 0.7174887892376681, 0, 0, 0.5094339622641509]
f1: [0.940279542566709, 0.9019607843137256, 0, 0.5776173285198556, 0, 0, 0.3552631578947368]
micro: (0.7566909975669099, 0.6327568667344863, 0.689196675900277)
macro: (0.390498787271244, 0.42378043286101047, 0.40330960923338166)


In [25]:
# label = ['ship','giá','chính hãng''','chất lượng','dịch vụ','an toàn','others']
# res = {'label':label,'p':X[0], 'r':X[1], 'f1':X[2]}
# df = pd.DataFrame(res)
# df1 = pd.DataFrame({'micro':X[3], 'macro': X[4]})
# final_df=pd.concat([df,df1], axis=1)

# final_df.to_csv(r"C:\Users\acer\Documents\hoc tap\khóa luận\mebe_domain\ket qua\bert_baseline.csv")